In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from pathlib import Path 
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle, gzip, math, matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor
import torch

In [6]:
def get_data():
    path = Path("/home/mayorov/.fastai/data/mnist.pkl.gz")
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")
    return map(tensor, (x_train, y_train, x_valid, y_valid))

In [7]:
def normalize(x, m, s):
    return (x-m)/s

In [8]:
x_train, y_train, x_valid, y_valid = get_data()

In [9]:
train_mean = x_train.mean()
train_std = x_train.std()
train_mean, train_std

(tensor(0.1304), tensor(0.3073))

In [10]:
x_train = normalize(x_train, train_mean, train_std)
# use train mean and std to normalize validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [11]:
# we want mean be close to 0 and std close to 1, lets check after normalizing
x_train.mean(), x_train.std()

(tensor(3.9162e-08), tensor(1.))

In [12]:
n, m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

## Foundations version

### Basic architecture

In [13]:
# hidden layer size 
nh = 50

In [43]:
# kaiming init
w1 = torch.randn(m, nh) * math.sqrt(2/m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, 1) * math.sqrt(2/nh)
b2 = torch.zeros(1)

In [37]:
def lin(x,w,b): return x@w+b

In [38]:
t = lin(x_valid, w1, b1)

In [39]:
t.mean(), t.std()

(tensor(-0.0089), tensor(1.4084))

In [40]:
def relu(x): return x.clamp_min(0.)

In [41]:
t = relu(lin(x_valid, w1, b1))

In [42]:
t.mean(), t.std()

(tensor(0.5503), tensor(0.8255))

In [44]:
from torch.nn import init

In [46]:
w1 = torch.zeros(m, nh)
init.kaiming_normal_(w1, mode="fan_out")
t = relu(lin(x_valid, w1, b1))c

In [47]:
t.mean(), t.std()

(tensor(0.4556), tensor(0.7165))

In [48]:
# just experiment
def relu(x): return x.clamp_min(0.) - 0.5

In [55]:
w1 = torch.zeros(m, nh)
init.kaiming_normal_(w1, mode="fan_out")
t = relu(lin(x_valid, w1, b1))

In [56]:
t.mean(), t.std()

(tensor(0.0184), tensor(0.7608))

In [57]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

### MSE loss

It is obviously does not make sense to use MSE in classification on MNIST, but we'll use it for simplicity for now

In [58]:
model(x_valid).shape

torch.Size([10000, 1])

In [63]:
# need to remove this ",1)" dimension, squeeze does it
def mse(output, targ): return (output.squeeze(1) - targ).pow(2).mean()

In [64]:
y_train, y_valid = y_train.float(), y_valid.float()

In [65]:
preds = model(x_train)

In [66]:
preds.shape

torch.Size([50000, 1])

In [67]:
mse(preds, y_train)

tensor(28.1191)

### Gradients and backward pass

In [77]:
def mse_grad(inp, targ):
    inp.g = 2. * (inp.squeeze()-targ).unsqueeze(1) / inp.shape[0]

In [78]:
def relu_grad(inp, out):
    inp.g = (inp>0).float() * out.g

In [79]:
def lin_grad(inp, out, w, b):
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1)*out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [80]:
def forward_and_backward(inp, targ):
    l1 = lin(inp, w1, b1)
    l2 = relu(l1)
    out = lin(l2, w2, b2)
    loss = mse(out, targ)
    
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1 )
    

In [81]:
forward_and_backward(x_train, y_train)

In [82]:
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig = x_train.g.clone()